In [101]:
import pandas as pd
import numpy as np
import json
import glob
import re

In [175]:
log_path = 'data/hand_eye_combined_logs'

dfs = []

for f in glob.glob(log_path + "/*_Normalized.txt"):
  d = pd.read_csv(f, sep="\s+")
  id = re.findall(".*Subject_(\d+)_.*", f)[0]
  d["id"] = id
  # forward/back fill NaN values to previous/next position 
  d['LeftY'] = d['LeftY'].ffill()
  d['LeftY'] = d['LeftY'].bfill()
  d['RightY'] = d['RightY'].ffill()
  d['RightY'] = d['RightY'].bfill()

  dfs.append(d)

df = pd.concat(dfs, ignore_index=True)

In [176]:
# look at time occurences across students – some students' sessions were longer than others
highest_time_occurence = max(df['TIME[s]'].value_counts())
df_common_time = df.groupby('TIME[s]').filter(lambda x: len(x) == highest_time_occurence)
df_common_time = df_common_time.iloc[1:,:].reset_index()

print(f"from %i to %i rows, a reduction of %.2f percent" % (len(df), len(df_common_time), (len(df) - len(df_common_time))/len(df)*100))
print(df['id'].value_counts())
print(df_common_time['id'].value_counts())

from 41676 to 27425 rows, a reduction of 34.19 percent
12    5639
11    4849
33    3549
07    3409
10    3289
06    3069
02    2639
03    2559
08    2559
04    2109
13    2029
15    2029
01    1989
14    1959
Name: id, dtype: int64
02    1959
03    1959
04    1959
11    1959
06    1959
13    1959
14    1959
01    1959
08    1959
15    1959
12    1959
07    1959
33    1959
10    1958
Name: id, dtype: int64


In [124]:
# calculate the euclidian distance between eye gaze and left/right hand
# Q: how do we engineer a feature that represents the coordination between both hands at once?
from math import sqrt
df['left_gaze_distance'] = ((df['LeftX'] - df['GazeX'])**2 + (df['LeftY'] - df['GazeY'])**2).apply(sqrt)
df['right_gaze_distance'] = ((df['RightX'] - df['GazeX'])**2 + (df['RightY'] - df['GazeY'])**2).apply(sqrt)

In [127]:
# turn the aggregated data frame into a one-student-per-row dataframe
left_df = df.pivot(index='id', columns='TIME[s]', values='left_gaze_distance').reset_index()
right_df = df.pivot(index='id', columns='TIME[s]', values='right_gaze_distance').reset_index()
left_df.columns.name=None
right_df.columns.name=None

# Have only one dataframe with both left and right coordinations as a tuple:
coordination_df = (pd.concat([left_df.loc[:, left_df.columns!='id'],right_df.loc[:, left_df.columns!='id']])
   .stack()
   .groupby(level=[0,1])
   .apply(tuple)
   .unstack()
 )

coordination_df['id'] = left_df['id']

left_df.set_index('id', inplace=True)
right_df.set_index('id', inplace=True)
coordination_df.set_index('id', inplace=True)

# Drop student one who had no data, and drop time 0.0 which had no data
coordination_df = coordination_df.iloc[1:, 1:]
left_df = left_df.iloc[1:, 1:]
right_df = right_df.iloc[1:, 1:]

coordination_df.head()

,0.2,0.3,0.4,0.5,0.6,0.7,0.8,0.9,1.0,1.1,...,194.9,195.0,195.1,195.2,195.3,195.4,195.5,195.6,195.7,195.8
id,,,,,,,,,,,,,,,,,,,,,
02,"(183.74166647769363, 209.1219739769114)","(38.91015291668744, 397.7863748295057)","(38.01315561749642, 398.72296146572745)","(36.235341863986875, 399.66235749692515)","(30.01666203960727, 398.2122549595881)","(367.98369529097346, 44.77722635447622)","(366.41506519246724, 37.8021163428716)","(363.72792029207767, 27.784887978899608)","(359.0891254271006, 30.528675044947494)","(398.0113063720678, 24.698178070456937)",...,"(141.4390328021229, 450.5873944086763)","(165.4962235218677, 440.33169315869145)","(319.2146613174276, 275.94383486499567)","(327.8673512260713, 251.5909378336191)","(336.04315199093105, 229.10696191953662)","(344.379151517626, 201.5365971728212)","(351.4498541755282, 178.22738285684386)","(539.4487927505261, 36.68787265568828)","(552.4137941796891, 38.27531841800928)","(557.0960419891709, 42.05948168962618)"
03,"(249.43536236869062, 258.15692901799093)","(26.870057685088806, 380.9580029347067)","(26.870057685088806, 380.888435109285)","(24.698178070456937, 381.8193290026056)","(20.09975124224178, 381.8193290026056)","(464.32100964742057, 95.524865872714)","(463.67445476325304, 94.79451460923254)","(463.38968482261237, 93.38094023943002)","(463.2763753959401, 93.38094023943002)","(463.1306511126207, 92.69843580125827)",...,"(10.770329614269007, 499.065126010624)","(12.36931687685298, 498.0461825975579)","(456.1633479358025, 43.01162633521314)","(456.1633479358025, 43.01162633521314)","(456.1633479358025, 43.01162633521314)","(39.21734310225516, 456.0800368356414)","(39.21734310225516, 456.0800368356414)","(39.21734310225516, 456.0800368356414)","(39.21734310225516, 456.0800368356414)","(39.21734310225516, 456.0800368356414)"
04,"(180.7373785358192,)","(184.43969204051496,)","(186.86893802876924,)","(374.73323845103465, 257.76927667974707)","(373.4059453195677, 258.3815008858026)","(370.1351104664349, 257.1089263327899)","(369.55784391621296, 257.1089263327899)","(368.7411015875502, 256.49366463910957)","(367.589444897429, 256.49366463910957)","(367.0163484097132, 255.55038642115179)",...,"(143.1258187749506, 333.2161460673837)","(138.63621460498698, 333.2161460673837)",NaN,NaN,"(145.12063946937388, 326.4659247149693)","(142.52368224263643, 326.4659247149693)","(184.36105879496353, 366.3249923223912)","(168.65645555388622, 366.3249923223912)","(118.00423721205946, 366.3249923223912)","(171.39719951037708, 445.7140338827127)"
06,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,"(633.7325934493191, 58.137767414994535)","(651.6011049714388, 119.20151005754919)","(651.6011049714388, 119.20151005754919)","(650.8002765826087, 120.06664815842908)","(650.0, 120.06664815842908)","(472.4595220757012, 297.83384629689084)","(461.7033679755867, 297.9463038871266)","(439.56796971571987, 298.0620740718282)","(644.2515036847024, 173.95401691251627)","(630.7598592174362, 174.89711261195822)"
07,"(205.34117950377123,)",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,"(369.4996617048519, 192.4162155328911)","(369.4996617048519, 192.4162155328911)","(369.4996617048519, 192.4162155328911)","(369.4996617048519, 192.4162155328911)","(369.8986347636336, 192.4162155328911)","(370.19589408852175, 192.4162155328911)","(370.19589408852175, 192.4162155328911)","(370.19589408852175, 192.77188591700815)","(368.982384403375, 192.77188591700815)","(522.2757892148553, 37.013511046643494)"


In [ ]:
# turn the aggregated data frame into a one-student-per-row dataframe
pivoted_df = df.pivot(index='id', columns='TIME[s]', values='left_gaze_distance').reset_index()
right_df = df.pivot(index='id', columns='TIME[s]', values='right_gaze_distance').reset_index()
left_df.columns.name=None
right_df.columns.name=None

In [97]:
# at this point, i am stuck because the left_df's values are continuous non-integers
coordination_num = len(df['left_gaze_distance'].unique())
coordination_array = preprocess(left_df, coordination_num)

IndexError: arrays used as indices must be of integer (or boolean) type

In [ ]:
coordination2coordination_model = build_coordination2coordination_model((99, coordination_num), lstm_dim=64)
coordination2coordination_model.fit(left_df[:, 0:-1], 
                      coordination_array[:, 1:],
                      epochs=20, 
                      batch_size=32, 
                      shuffle=True,
                      validation_split=0.2)

In [177]:
df_common_time['Y_proportion'] = df_common_time['RightY'] / df_common_time['LeftY']

In [214]:
# approach: bin time series data based on % time in green for a window of time
# use rnn to predict next % in green given sequence of previous % in green

window = 10 # window of 10 ==> 1 second
rows = []

for group_id, group_df in df_common_time.groupby(['id']):
    start = 0

    for i in range(int(len(group_df) / window)):
      slice = group_df.iloc[start:start+window]
      avg_proportion = slice['Y_proportion'].mean()

      t_start = group_df.iloc[start]['TIME[s]']
      t_end = group_df.iloc[start+window]['TIME[s]']
      if(group_id == '10'):
          t_start = np.ceil(t_start - .1)
          t_end = np.ceil(t_end - .1)

      time_range = f"%s_%s" % (t_start, t_end)

      row = [
          group_id, 
          time_range,
          avg_proportion
          ]

      rows.append(row)
      start = start + window

pro_df = pd.DataFrame(rows,
    columns=[
        'id', 
        'time_window', 
        "avg_proportion"
        ])


In [280]:
a, b = pro_df_pivot.loc[:, pro_df_pivot.columns!='id'].stack().idxmin()
min_pro = pro_df_pivot.loc[[a], [b]]
min_pro = min_pro.loc[min_pro.index[0]][0]

a, b = pro_df_pivot.loc[:, pro_df_pivot.columns!='id'].stack().idxmax()
max_pro = pro_df_pivot.loc[[a], [b]]
max_pro = max_pro.loc[max_pro.index[0]][0]

In [247]:
pro_df['pro_diff'] = np.abs(2 - pro_df['avg_proportion'])

In [300]:
def assign_bin(pro_diff):
  if(pro_diff) < .01: return 0
  if(pro_diff) < .03: return 1
  if(pro_diff) < .05: return 2
  if(pro_diff) < .7: return 3
  elif(pro_diff) < 1: return 4
  elif(pro_diff) < 2: return 5
  elif(pro_diff) < 3: return 6
  else: return 7

In [301]:
pro_df['pro_bin'] = pro_df['pro_diff'].apply(assign_bin)
pro_df['pro_bin'].value_counts()

3    809
5    758
4    432
1    307
0    185
2    179
7     43
6     17
Name: pro_bin, dtype: int64

In [302]:
# turn the aggregated data frame into a one-student-per-row dataframe
pro_df_pivot = pro_df.pivot(index='id', columns='time_window', values='pro_bin').reset_index()
pro_df_pivot.columns.name=None

pro_df_pivot

,id,0.0_1.0,1.0_2.0,10.0_11.0,100.0_101.0,101.0_102.0,102.0_103.0,103.0_104.0,104.0_105.0,105.0_106.0,...,90.0_91.0,91.0_92.0,92.0_93.0,93.0_94.0,94.0_95.0,95.0_96.0,96.0_97.0,97.0_98.0,98.0_99.0,99.0_100.0
0,01,5,5,4,3,3,5,4,4,5,...,1,1,1,3,3,3,3,5,5,4
1,02,4,4,5,5,5,5,5,5,5,...,5,5,5,5,5,5,5,5,5,5
2,03,5,5,5,5,5,5,5,5,5,...,3,5,5,4,4,4,4,4,4,4
3,04,5,5,5,4,5,3,3,3,5,...,4,4,5,4,4,4,5,5,4,4
4,06,3,5,3,3,1,0,0,2,3,...,3,3,3,3,5,3,4,4,3,3
5,07,5,5,5,5,4,4,3,3,2,...,4,5,5,5,5,5,5,5,5,5
6,08,5,4,3,3,3,3,3,3,3,...,2,0,0,1,1,1,1,3,4,5
7,10,4,4,0,1,1,0,3,3,4,...,3,0,1,3,1,1,1,1,1,1
8,11,4,4,5,5,5,5,5,4,3,...,7,7,7,7,7,5,5,5,4,4
9,12,5,5,4,3,3,3,0,1,1,...,0,3,3,3,3,3,3,3,3,3


In [305]:
from keras.layers import Input, Dense, LSTM, TimeDistributed, Lambda, multiply
from keras.models import Model
from tensorflow.keras.optimizers import RMSprop, Adam
from keras.preprocessing.sequence import pad_sequences
from keras import backend as K

def build_s2s_model(input_shape, lstm_dim=32, dropout=0.0):
    input = Input(shape=input_shape, name='input coordinations')
    lstm = LSTM(lstm_dim, 
                return_sequences=True, 
                dropout=dropout,
                name='lstm_layer')(input)
    output = TimeDistributed(Dense(input_shape[-1], activation='softmax'), name='probability')(lstm)
    model = Model(inputs=[input], outputs=[output])
    adam = Adam(learning_rate=0.001, beta_1=0.9, beta_2=0.999, epsilon=1e-07, decay=0.0)
    model.compile(optimizer=adam, loss='categorical_crossentropy', metrics=['accuracy'])
    model.summary()
    return model

In [308]:
s2s_model = build_s2s_model((194, num_bins), lstm_dim=64)

Model: "model_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input coordinations (InputL  [(None, 194, 8)]         0         
 ayer)                                                           
                                                                 
 lstm_layer (LSTM)           (None, 194, 64)           18688     
                                                                 
 probability (TimeDistribute  (None, 194, 8)           520       
 d)                                                              
                                                                 
Total params: 19,208
Trainable params: 19,208
Non-trainable params: 0
_________________________________________________________________


In [ ]:
def preprocess(df, num_bins):
  matrix = df.iloc[:, 1:].values
  num_bins = num_bins

  seq_len = matrix.shape[1]
  one_hot_array = np.zeros((matrix.shape[0], seq_len, num_bins))
  for i in range(matrix.shape[0]):
      one_hot_array[i, np.arange(seq_len), matrix[i]] = 1
  return one_hot_array

In [313]:
num_bins = len(pro_df['pro_bin'].unique())

In [317]:
# 70% of data used as training data
test_prop = 0.7

# Split user_ids/student_ids into 70% training and 30% test data
idx_split = np.array(pro_df_pivot['id'].unique())
np.random.seed(42)
np.random.shuffle(idx_split)
train_ids = idx_split[:int(test_prop * len(idx_split))]
test_ids = idx_split[int(test_prop * len(idx_split)):]

# Split skill_df
train_df = pro_df_pivot.loc[pro_df_pivot['id'].isin(train_ids)].reset_index(drop = True)
test_df = pro_df_pivot.loc[pro_df_pivot['id'].isin(test_ids)].reset_index(drop = True)

# Checking that the ratio of training data to test data is adequate
print('Training data is %5.3f' % (train_df.shape[0]/pro_df_pivot.shape[0]),'% of total data')
print('Testing data is %5.3f' % (test_df.shape[0]/pro_df_pivot.shape[0]),'% of total data')

train_array = preprocess(train_df, num_bins)
test_array = preprocess(test_df, num_bins)


Training data is 0.643 % of total data
Testing data is 0.357 % of total data


In [315]:
s2s_model.fit(train_array[:, 0:-1], 
                      train_array[:, 1:],
                      epochs=20, 
                      batch_size=32, 
                      shuffle=True,
                      validation_split=0.2)

Epoch 1/20
1/1 [==============================] - 2s 2s/step - loss: 1.5408 - accuracy: 0.4043 - val_loss: 1.7365 - val_accuracy: 0.3325
Epoch 2/20
1/1 [==============================] - 0s 91ms/step - loss: 1.5363 - accuracy: 0.4043 - val_loss: 1.7405 - val_accuracy: 0.3351
Epoch 3/20
1/1 [==============================] - 0s 106ms/step - loss: 1.5325 - accuracy: 0.4035 - val_loss: 1.7410 - val_accuracy: 0.3247
Epoch 4/20
1/1 [==============================] - 0s 102ms/step - loss: 1.5288 - accuracy: 0.4124 - val_loss: 1.7357 - val_accuracy: 0.3299
Epoch 5/20
1/1 [==============================] - 0s 83ms/step - loss: 1.5238 - accuracy: 0.4124 - val_loss: 1.7248 - val_accuracy: 0.3325
Epoch 6/20
1/1 [==============================] - 0s 77ms/step - loss: 1.5175 - accuracy: 0.4138 - val_loss: 1.7108 - val_accuracy: 0.3376
Epoch 7/20
1/1 [==============================] - 0s 84ms/step - loss: 1.5110 - accuracy: 0.4146 - val_loss: 1.6970 - val_accuracy: 0.3479
Epoch 8/20
1/1 [===========

In [318]:
# Make a prediction on testing data
test_predict = s2s_model.predict(test_array[:, 0:-1])
print("Shape of prediction on test set:", test_predict.shape, ' = (student, time step, proportion) \n')

# Evaulate model prediction accuracy on test data
results = s2s_model.evaluate(test_array[:, 0:-1], test_array[:, 1:])
print('\nThe overall accuracy of skill prediction in the test set is: %5.2f' % (100*results[1]),'%')

2022-04-30 17:50:48.596749: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:113] Plugin optimizer for device_type GPU is enabled.
2022-04-30 17:50:48.648544: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:113] Plugin optimizer for device_type GPU is enabled.


Shape of prediction on test set: (5, 194, 8)  = (student, time step, proportion) 

1/1 [==============================] - 0s 283ms/step - loss: 1.5006 - accuracy: 0.4412

The overall accuracy of skill prediction in the test set is: 44.12 %


In [319]:
models = []

for lstm_dim in [64, 128]:
    for batch_size in [8, 16]:
        for validation_split in [0, 0.1, 0.2, 0.3]:
            s2s_model = build_s2s_model((194, num_bins), lstm_dim=lstm_dim)
            
            s2s_model.fit(train_array[:, 0:-1], 
                      train_array[:, 1:],
                      epochs=20, 
                      batch_size=batch_size, 
                      shuffle=True,
                      validation_split=validation_split, verbose=0)

            # Evaulate model prediction accuracy on test data
            results = s2s_model.evaluate(test_array[:, 0:-1], test_array[:, 1:])

            models.append({'lstm_dim': lstm_dim, 'batch_size': batch_size, 'validation_split': validation_split, 'accuracy': 100*results[1]}) 

Model: "model_2"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input coordinations (InputL  [(None, 194, 8)]         0         
 ayer)                                                           
                                                                 
 lstm_layer (LSTM)           (None, 194, 64)           18688     
                                                                 
 probability (TimeDistribute  (None, 194, 8)           520       
 d)                                                              
                                                                 
Total params: 19,208
Trainable params: 19,208
Non-trainable params: 0
_________________________________________________________________


2022-04-30 17:57:00.241915: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:113] Plugin optimizer for device_type GPU is enabled.
2022-04-30 17:57:00.426902: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:113] Plugin optimizer for device_type GPU is enabled.
2022-04-30 17:57:00.896694: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:113] Plugin optimizer for device_type GPU is enabled.
2022-04-30 17:57:06.091704: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:113] Plugin optimizer for device_type GPU is enabled.
2022-04-30 17:57:06.143582: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:113] Plugin optimizer for device_type GPU is enabled.


1/1 [==============================] - 0s 419ms/step - loss: 1.6217 - accuracy: 0.3897
Model: "model_3"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input coordinations (InputL  [(None, 194, 8)]         0         
 ayer)                                                           
                                                                 
 lstm_layer (LSTM)           (None, 194, 64)           18688     
                                                                 
 probability (TimeDistribute  (None, 194, 8)           520       
 d)                                                              
                                                                 
Total params: 19,208
Trainable params: 19,208
Non-trainable params: 0
_________________________________________________________________


2022-04-30 17:57:07.130296: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:113] Plugin optimizer for device_type GPU is enabled.
2022-04-30 17:57:07.261193: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:113] Plugin optimizer for device_type GPU is enabled.
2022-04-30 17:57:07.550213: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:113] Plugin optimizer for device_type GPU is enabled.
2022-04-30 17:57:08.240791: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:113] Plugin optimizer for device_type GPU is enabled.
2022-04-30 17:57:08.291132: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:113] Plugin optimizer for device_type GPU is enabled.


1/1 [==============================] - 0s 210ms/step - loss: 1.7274 - accuracy: 0.2732
Model: "model_4"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input coordinations (InputL  [(None, 194, 8)]         0         
 ayer)                                                           
                                                                 
 lstm_layer (LSTM)           (None, 194, 64)           18688     
                                                                 
 probability (TimeDistribute  (None, 194, 8)           520       
 d)                                                              
                                                                 
Total params: 19,208
Trainable params: 19,208
Non-trainable params: 0
_________________________________________________________________


...

1/1 [==============================] - 0s 223ms/step - loss: 1.6794 - accuracy: 0.3474
Model: "model_16"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input coordinations (InputL  [(None, 194, 8)]         0         
 ayer)                                                           
                                                                 
 lstm_layer (LSTM)           (None, 194, 128)          70144     
                                                                 
 probability (TimeDistribute  (None, 194, 8)           1032      
 d)                                                              
                                                                 
Total params: 71,176
Trainable params: 71,176
Non-trainable params: 0
_________________________________________________________________


2022-04-30 17:58:11.050565: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:113] Plugin optimizer for device_type GPU is enabled.
2022-04-30 17:58:11.191193: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:113] Plugin optimizer for device_type GPU is enabled.
2022-04-30 17:58:11.502677: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:113] Plugin optimizer for device_type GPU is enabled.
2022-04-30 17:58:12.220214: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:113] Plugin optimizer for device_type GPU is enabled.
2022-04-30 17:58:12.275855: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:113] Plugin optimizer for device_type GPU is enabled.


1/1 [==============================] - 0s 245ms/step - loss: 1.6602 - accuracy: 0.3835
Model: "model_17"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input coordinations (InputL  [(None, 194, 8)]         0         
 ayer)                                                           
                                                                 
 lstm_layer (LSTM)           (None, 194, 128)          70144     
                                                                 
 probability (TimeDistribute  (None, 194, 8)           1032      
 d)                                                              
                                                                 
Total params: 71,176
Trainable params: 71,176
Non-trainable params: 0
_________________________________________________________________


2022-04-30 17:58:15.043393: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:113] Plugin optimizer for device_type GPU is enabled.
2022-04-30 17:58:15.178212: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:113] Plugin optimizer for device_type GPU is enabled.
2022-04-30 17:58:15.454635: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:113] Plugin optimizer for device_type GPU is enabled.
2022-04-30 17:58:16.117115: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:113] Plugin optimizer for device_type GPU is enabled.
2022-04-30 17:58:16.172774: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:113] Plugin optimizer for device_type GPU is enabled.


1/1 [==============================] - 0s 228ms/step - loss: 1.6502 - accuracy: 0.4268


In [324]:
sorted(models, key=lambda d: d['accuracy'], reverse=True) 

[{'lstm_dim': 128,
  'batch_size': 8,
  'validation_split': 0,
  'accuracy': 43.09278428554535},
 {'lstm_dim': 128,
  'batch_size': 8,
  'validation_split': 0.1,
  'accuracy': 42.98969209194183},
 {'lstm_dim': 128,
  'batch_size': 8,
  'validation_split': 0.2,
  'accuracy': 42.7835077047348},
 {'lstm_dim': 128,
  'batch_size': 16,
  'validation_split': 0.3,
  'accuracy': 42.68041253089905},
 {'lstm_dim': 128,
  'batch_size': 8,
  'validation_split': 0.3,
  'accuracy': 42.061856389045715},
 {'lstm_dim': 128,
  'batch_size': 16,
  'validation_split': 0,
  'accuracy': 40.30928015708923},
 {'lstm_dim': 64,
  'batch_size': 8,
  'validation_split': 0,
  'accuracy': 38.969072699546814},
 {'lstm_dim': 128,
  'batch_size': 16,
  'validation_split': 0.2,
  'accuracy': 38.35051655769348},
 {'lstm_dim': 128,
  'batch_size': 16,
  'validation_split': 0.1,
  'accuracy': 34.74226891994476},
 {'lstm_dim': 64,
  'batch_size': 16,
  'validation_split': 0.3,
  'accuracy': 33.814433217048645},
 {'lstm_dim